In [18]:
from selenium import webdriver
import numpy as np
import glob
import os
import datetime
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
d = webdriver.Chrome()
d.get("https://bitinfocharts.com/")

In [10]:
table = d.find_element_by_id('main_body').find_element_by_tag_name('tbody').get_attribute('innerHTML')
bs = BeautifulSoup(table, "lxml")

In [146]:
df = pd.DataFrame()
df['Columns'] = [k.find('td').getText() for k in bs.find_all('tr')]
for crypto in ['btc', 'eth', 'ltc']:
    df[crypto.upper()] = [p.getText() for p in bs.find_all(class_='c_%s' % crypto)]

In [147]:
stopwords = ['BTC', 'ETH', 'LTC']
def remove_chars(x):
    num = re.sub('[^0-9]','', x)
    if len(str(num)) < 3:
        return x
    else:
        return num
    
def convert_number(x):
    return float(x.replace(',',''))

def remove_crypt(x):
    for k in stopwords:
        if k in x:
            return convert_number(x[:x.index(x)])

df.loc[1] = df.loc[1].apply(remove_chars)
df.loc[2] = df.loc[2].apply(lambda x:convert_number(x[x.find('$')+1:x.find('USD')]) if 'USD' in x else x)
df.loc[3] = df.loc[3].apply(lambda x:convert_number(x[x.find('$')+1:x.find('USD')]) if 'USD' in x else x)
df.iloc[4, 1:] = df.iloc[4, 1:].apply(convert_number)
for p in range(6,10):
    df.iloc[p, 1:] = df.iloc[p, 1:].apply(remove_crypt)

ValueError: could not convert string to float: 

In [130]:
df.iloc[6:9]

,Columns,BTC,ETH,LTC
6,Sent last 24h,"3,952,471 BTC ($29,296,466,364 USD) 23.71% mar...","14,454,733 ETH ($4,746,058,411 USD) 15.09% mar...","22,743,431 LTC ($1,439,506,319 USD) 42.23% mar..."
7,Sent avg. per hour,"164,686 BTC ($1,220,686,098 USD)","602,281 ETH ($197,752,434 USD)","947,643 LTC ($59,979,430 USD)"
8,Avg. Transaction Value,"12.60 BTC ($93,411 USD)","28.86 ETH ($9,476 USD)","876.23 LTC ($55,459 USD)"


In [125]:
df.head(15)

,Columns,BTC,ETH,LTC
0,,"Bitcoin(explorer, top100)",Ethereum,"Litecoin(explorer, top100)"
1,Total,16669774,95762051,53850732
2,Price,7412.19,328.34,63.29
3,Market Capitalization,1.2356e+11,3.14425e+10,3.40839e+09
4,Transactions last 24h,313631,500844,25956
5,Transactions avg. per hour,"13,068","20,869","1,082"
6,Sent last 24h,"3,952,471 BTC ($29,296,466,364 USD) 23.71% mar...","14,454,733 ETH ($4,746,058,411 USD) 15.09% mar...","22,743,431 LTC ($1,439,506,319 USD) 42.23% mar..."
7,Sent avg. per hour,"164,686 BTC ($1,220,686,098 USD)","602,281 ETH ($197,752,434 USD)","947,643 LTC ($59,979,430 USD)"
8,Avg. Transaction Value,"12.60 BTC ($93,411 USD)","28.86 ETH ($9,476 USD)","876.23 LTC ($55,459 USD)"
9,Median Transaction Value,0.096 BTC ($714.56 USD),0.052 ETH ($17.2 USD),6.06 LTC ($383.79 USD)


In [116]:
df.iloc[4, 1:]

BTC    313,631
ETH    500,844
LTC     25,956
Name: 4, dtype: object